In [ ]:
import zipfile
from pathlib import Path
import partitura
import pandas as pd
import xml.etree.ElementTree as ET
from spacy import displacy
import numpy as np

/share/home/francesco/miniconda3/envs/musicparser/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'spacy'

In [ ]:
# delete all mus files in all subfolders
for file in Path('data').rglob("*"):
    if file.suffix == '.mus' or file.suffix == '.mid' or file.suffix == '.kern':
        file.unlink()

In [ ]:
#unzip all files in zips folder

# for zip_file in Path("zips").iterdir():
#     with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#         zip_ref.extractall(Path("unzipped"))

In [ ]:
# check if ts trees are in unzipped folder
# for folder in Path("unzipped").iterdir():
#     if not any([file.name.startswith("TS") for file in folder.iterdir()]):
#         print("not a TS tree in ", folder)

In [ ]:
# check if there are scores everywhere
# for folder in Path("unzipped").iterdir():
#     msc_file = [file for file in folder.iterdir() if file.name.startswith("MSC")]
#     numbered_file = [file for file in folder.iterdir() if file.name.startswith(folder.name[:2])]
#     assert(len(msc_file)+len(numbered_file) == 1)
#     score_file = msc_file[0] if len(msc_file) == 1 else numbered_file[0]
#     try:
#         score = partitura.load_musicxml(str(score_file))
#     except Exception as e:
#         print("error loading ", score_file, e)

In [ ]:
# build a pandas dataframe with all the scores paths
def get_score_path(folder):
    score_files = [file for file in folder.iterdir() if file.name.startswith("score")]
    assert(len(score_files) == 1)
    return str(score_files[0])

def get_ts_path(folder):
    ts_file = [file for file in folder.iterdir() if file.name.startswith("TS")]
    assert(len(ts_file) == 1)
    return str(ts_file[0])

def get_title(folder):
    title_file = [file for file in folder.iterdir() if file.suffix == ".txt"]
    assert(len(title_file) == 1)
    return title_file[0].name

list_of_tuples = []

for folder in Path("data").iterdir():
    list_of_tuples.append((get_title(folder),get_score_path(folder), get_ts_path(folder)))

df = pd.DataFrame(list_of_tuples, columns = ['title', 'score', 'ts'])

In [ ]:
df

,title,score,ts
0,01_Waltz in E flat Grande Valse Brillante Op.1...,data\01\score.xml,data\01\TS.xml
1,02_Moments Musicaux.txt,data\02\score.xml,data\02\TS.xml
2,03_Bagatelle 'Fur Elise' WoO.59.txt,data\03\score.xml,data\03\TS.xml
3,04_The Preludes Op.28 No.15.txt,data\04\score.xml,data\04\TS.xml
4,05_Turkish March.txt,data\05\score.xml,data\05\TS.xml
...,...,...,...
295,95_12 Variationen uber ein franzosisches Lied ...,data\95\score.xml,data\95\TS.xml
296,96_Lieder ohne Worte Heft 5 Op.62-6 Fruhlingsl...,data\96\score.xml,data\96\TS.xml
297,97_Les Patineurs Op.183.txt,data\97\score.xml,data\97\TS.xml
298,98_Sonate fur Klavier Nr.8 c moll Pathetique O...,data\98\score.xml,data\98\TS.xml


# Parse tree annotations from the xml file

In [ ]:
# parse the xml file with annotations in a list of dependencies
tree = ET.parse(r'data\07\TS-07.xml')
root = tree.getroot()

In [ ]:
# def iterative_parse(xml_elem):
#     primary_children = xml_elem.find("ts").find("primary")
#     secondary_children = xml_elem.find("ts").find("secondary")
#     # recursion ending condition
#     if primary_children is None:
#         assert secondary_children is None
#         return xml_elem.find("ts").find("head").find("chord").find("note").attrib["id"]
#     else:
#         assert secondary_children is not None
#         return [iterative_parse(primary_children), iterative_parse(secondary_children)]


# iterative_parse(root)

In [ ]:
def iterative_parse(xml_elem):
    primary_children = xml_elem.find("ts").find("primary")
    secondary_children = xml_elem.find("ts").find("secondary")
    if primary_children is None: # recursion ending condition
        assert secondary_children is None
        return [], xml_elem.find("ts").find("head").find("chord").find("note").attrib["id"]
    else: # recursive call
        assert secondary_children is not None
        out_list = [] # dependency list
        iterative_result_primary = iterative_parse(primary_children)
        iterative_result_secondary = iterative_parse(secondary_children)
        # merge the dependencies lists computed deeper
        out_list.extend(iterative_result_primary[0])
        out_list.extend(iterative_result_secondary[0])
        # append the dependency for the current node
        out_list.append((iterative_result_primary[1], iterative_result_secondary[1]))
        # return the dependency list, and the id of the current node, i.e., the primary
        return out_list, iterative_parse(primary_children)[1]


arcs = iterative_parse(root)[0]
arcs

[('P1-9-5', 'P1-9-4'),
 ('P1-9-7', 'P1-9-6'),
 ('P1-9-5', 'P1-9-7'),
 ('P1-9-9', 'P1-9-8'),
 ('P1-9-5', 'P1-9-9'),
 ('P1-9-10', 'P1-9-5'),
 ('P1-9-1', 'P1-9-2'),
 ('P1-9-1', 'P1-9-3'),
 ('P1-9-10', 'P1-9-1'),
 ('P1-8-5', 'P1-8-4'),
 ('P1-8-6', 'P1-8-5'),
 ('P1-8-2', 'P1-8-3'),
 ('P1-8-1', 'P1-7-15'),
 ('P1-8-2', 'P1-8-1'),
 ('P1-8-6', 'P1-8-2'),
 ('P1-9-10', 'P1-8-6'),
 ('P1-6-1', 'P1-5-12'),
 ('P1-6-9', 'P1-6-8'),
 ('P1-6-7', 'P1-6-6'),
 ('P1-6-2', 'P1-6-3'),
 ('P1-6-4', 'P1-6-5'),
 ('P1-6-2', 'P1-6-4'),
 ('P1-6-7', 'P1-6-2'),
 ('P1-6-9', 'P1-6-7'),
 ('P1-6-1', 'P1-6-9'),
 ('P1-6-15', 'P1-6-14'),
 ('P1-6-12', 'P1-6-13'),
 ('P1-6-11', 'P1-6-12'),
 ('P1-6-15', 'P1-6-11'),
 ('P1-7-1', 'P1-6-15'),
 ('P1-6-1', 'P1-7-1'),
 ('P1-7-13', 'P1-7-12'),
 ('P1-7-14', 'P1-7-13'),
 ('P1-7-3', 'P1-7-2'),
 ('P1-7-5', 'P1-7-4'),
 ('P1-7-3', 'P1-7-5'),
 ('P1-7-11', 'P1-7-10'),
 ('P1-7-7', 'P1-7-6'),
 ('P1-7-9', 'P1-7-8'),
 ('P1-7-7', 'P1-7-9'),
 ('P1-7-11', 'P1-7-7'),
 ('P1-7-3', 'P1-7-11'),
 ('P1-7-14',

In [ ]:
import numpy as np
note_ids = list(np.unique([e.attrib['id'] for e in root.findall('''.//note''')]))

In [ ]:
spacy_words = [{"text": word, "tag": ""} for word in note_ids]
spacy_arcs = []
for (start_id, end_id) in arcs:
    start_ix = note_ids.index(start_id)
    end_ix = note_ids.index(end_id)
    if start_ix < end_ix:
        spacy_arcs.append({"start": start_ix, "end": end_ix, "label": "", "dir": "right"})
    else:
        spacy_arcs.append({"start": end_ix, "end": start_ix, "label": "", "dir": "left"})


spacy_dict = {"words": spacy_words, "arcs": spacy_arcs}
displacy.render(spacy_dict, style='dep', jupyter=True, manual=True)

In [ ]:
import spacy
from spacy import displacy

# Load the language model
nlp = spacy.load("en_core_web_sm")

sentence = 'Deemed universities charge huge fees'

# nlp function returns an object with individual token information, 
# linguistic features and relationships
doc = nlp(sentence)

print ("{:<15} | {:<8} | {:<15} | {:<20}".format('Token','Relation','Head', 'Children'))
print ("-" * 70)

for token in doc:
  # Print the token, dependency nature, head and all dependents of the token
  print ("{:<15} | {:<8} | {:<15} | {:<20}"
         .format(str(token.text), str(token.dep_), str(token.head.text), str([child for child in token.children])))
  
# Use displayCy to visualize the dependency 
displacy.render(doc, style='dep', jupyter=True, options={'distance': 120})

Token           | Relation | Head            | Children            
----------------------------------------------------------------------
Deemed          | amod     | universities    | []                  
universities    | nsubj    | charge          | [Deemed]            
charge          | ROOT     | charge          | [universities, fees]
huge            | amod     | fees            | []                  
fees            | dobj     | charge          | [huge]              


In [ ]:
l = []
for token in doc:
    l.append(token)
type(doc)

spacy.tokens.doc.Doc

In [ ]:
obj = {
    "words": [
        {"text": "This", "tag": ""},
        {"text": "is", "tag": ""},
        {"text": "a", "tag": ""},
        {"text": "sentence", "tag": ""}
    ],
    "arcs": [
        {"start": 0, "end": 1, "label": "", "dir": "right"},
        {"start": 2, "end": 3, "label": "", "dir": "left"},
        {"start": 1, "end": 3, "label": "", "dir": "right"}
    ]
}

displacy.render(obj, style='dep', jupyter=True, options={'distance': 120}, manual=True)

# Parse note features from score file

In [ ]:
score_path = df.iloc[0]["score"]

In [ ]:
score = partitura.load_musicxml(score_path)
len(score.parts[0].rests)

0

In [ ]:
extended_score_note_array = partitura.utils.music.ensure_notearray(
    score,
    include_metrical_position=True, # adds 3 fields: is_downbeat, rel_onset_div, tot_measure_div
    include_time_signature=True, # adds 2 fields: time_signature_numerator, time_signature_denominator
)

In [ ]:
np.char.array(extended_score_note_array["ts_beats"]) + np.char.array(extended_score_note_array["ts_beat_type"])

chararray([b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34',
           b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34',
           b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34',
           b'34'], dtype='|S2')

In [ ]:
scores = [partitura.load_musicxml(score_path) for score_path in df["score"]]  
    

c:\Users\fosca\anaconda3\envs\mir_research\lib\site-packages\partitura\directions.py:533: UserWarning: error parsing "sentito" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
c:\Users\fosca\anaconda3\envs\mir_research\lib\site-packages\partitura\directions.py:533: UserWarning: error parsing "Largamente" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
c:\Users\fosca\anaconda3\envs\mir_research\lib\site-packages\partitura\directions.py:533: UserWarning: error parsing "cresc.  -   -   -  " (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
c:\Users\fosca\anaconda3\envs\mir_research\lib\site-packages\partitura\directions.py:533: UserWarning: error parsing "cre    -     -     -     scen     -    -    -     do" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
c:\Users\fosca\anaconda3\envs\mir_re

In [ ]:
all_time_signatures = []

for i,score in enumerate(scores):
    na = partitura.utils.music.ensure_notearray(
        score,
        include_metrical_position=True,
        include_time_signature=True,        
    )
    # if len(score.parts[0].rests) > 0:
    #     print(score_path, len(score.parts[0].rests))
    time_signatures = np.char.array(na["ts_beats"].astype(str)) + np.char.array(["/"]*na.shape[0])+ np.char.array(na["ts_beat_type"].astype(str))
    all_time_signatures.append(np.unique(time_signatures)[-1])
    if np.unique(time_signatures).shape[0] > 1:
        print(np.unique(time_signatures), df["score"][i], np.unique(time_signatures)[-1] )

['1/8' '2/8' '3/8'] data\03\03_Bagatelle 'Fur Elise' WoO.59.xml 3/8
['1/8' '5/8' '6/8'] data\06\06_Blumenlied Op.39.xml 6/8
['1/8' '11/8' '12/8'] data\07\07_Nocturne.xml 12/8
['1/4' '3/4' '4/4'] data\09\09_String Quartet in F major Op.3 No.5 Serenade.xml 4/4
['1/4' '2/4' '3/4'] data\10\10_Wiegenlied.xml 3/4
['1/4' '4/4'] data\11\11_Solvejgs Lied.xml 4/4
['2/4' '3/4'] data\115\MSC-115.xml 3/4
['1/4' '2/4' '3/4'] data\12\12_Anitras Dans.xml 3/4
['1/4' '3/4' '4/4'] data\13\13_Traumerei.xml 4/4
['1/4' '2/4' '3/4'] data\14\14_Menuett No.2 in G maj.xml 3/4
['1/4' '3/4' '4/4'] data\16\16_William Tell Overture.xml 4/4
['1/4' '2/4' '3/4'] data\19\19_Tannhauser Overture.xml 3/4
['1/8' '2/8' '3/8'] data\20\20_La Traviata Brindisi.xml 3/8
['1/8' '5/8' '6/8'] data\21\21_Plaisir d'Amour.xml 6/8
['2/4' '4/4'] data\25\25_L'arlesienne Suite No.2 Farandole.xml 4/4
['1/4' '2/2' '3/4'] data\37\37_Sonate fur Klavier Nr.48 C dur Op.30-1 Mov.1.xml 3/4
['1/4' '2/4' '3/4'] data\49\49_The Planets Op.32 Jupiter,

In [ ]:
np.unique(all_time_signatures)
# for i, ts in enumerate(all_time_signatures):
#     print(ts, df["score"][i])

array(['12/8', '2/2', '2/4', '3/2', '3/4', '3/8', '4/4', '6/4', '6/8',
       '9/8'], dtype='<U4')

In [ ]:
np.char.array([","]*na.shape[0]) + np.char.array(na["ts_beat_type"].astype(str))

chararray([',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4',
           ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4',
           ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4'], dtype='<U12')

In [ ]:
score_path = df.iloc[6]["score"]
score = partitura.load_musicxml(score_path)

In [ ]:
na = partitura.utils.music.ensure_notearray(
        score,
        include_metrical_position=True,
        include_time_signature=True,    
    )

na, real_ts,real_measure_duration = correct_metrical_information(na)

na
# na

array([( 0. , 1. ,  0.  , 0.5 ,   0,  2, 70, 1, 'None', 12, 8, 12, 1, 22, 24, 4),
       ( 1. , 4. ,  0.5 , 2.  ,   2,  8, 79, 1, 'None', 12, 8, 12, 1,  0, 24, 4),
       ( 5. , 1. ,  2.5 , 0.5 ,  10,  2, 77, 1, 'None', 12, 8, 12, 0,  8, 24, 4),
       ( 6. , 1. ,  3.  , 0.5 ,  12,  2, 79, 1, 'None', 12, 8, 12, 0, 10, 24, 4),
       ( 7. , 3. ,  3.5 , 1.5 ,  14,  6, 77, 1, 'None', 12, 8, 12, 0, 12, 24, 4),
       (10. , 2. ,  5.  , 1.  ,  20,  4, 75, 1, 'None', 12, 8, 12, 0, 18, 24, 4),
       (12. , 1. ,  6.  , 0.5 ,  24,  2, 70, 1, 'None', 12, 8, 12, 0, 22, 24, 4),
       (13. , 2. ,  6.5 , 1.  ,  26,  4, 79, 1, 'None', 12, 8, 12, 1,  0, 24, 4),
       (15. , 1. ,  7.5 , 0.5 ,  30,  2, 72, 1, 'None', 12, 8, 12, 0,  4, 24, 4),
       (16. , 2. ,  8.  , 1.  ,  32,  4, 84, 1, 'None', 12, 8, 12, 0,  6, 24, 4),
       (18. , 1. ,  9.  , 0.5 ,  36,  2, 79, 1, 'None', 12, 8, 12, 0, 10, 24, 4),
       (19. , 3. ,  9.5 , 1.5 ,  38,  6, 82, 1, 'None', 12, 8, 12, 0, 12, 24, 4),
       (22. , 2.

In [ ]:
def correct_metrical_information(na):
    time_signatures = np.char.array(na["ts_beats"].astype(str)) + np.char.array(["/"]*na.shape[0])+ np.char.array(na["ts_beat_type"].astype(str))
    # get real time signature and measure duration, discarding pickup and ending measure ts
    real_ts = np.unique(time_signatures)[-1]
    real_measure_duration = na[time_signatures == real_ts][0]["tot_measure_div"]
    # find pickup notes
    pickup_note_indices = np.where((na["tot_measure_div"]!= real_measure_duration) * (na["onset_div"] < real_measure_duration ))[0]
    # set the pickup note to the correct metrical position
    na["rel_onset_div"][pickup_note_indices] = na["onset_div"][pickup_note_indices] + real_measure_duration - na["tot_measure_div"][pickup_note_indices]
    # set the measure duration to correct value
    na["tot_measure_div"] = real_measure_duration
    # set the correct time signature
    na["ts_beats"] = real_ts.split("/")[0]
    na["ts_mus_beats"] = real_ts.split("/")[0]
    na["ts_beat_type"] = real_ts.split("/")[1]
    return na, real_ts, real_measure_duration

def get_note_features(na):
    pitch = na["pitch"]
    metrical =  na["is_downbeat"] #TODO: add more metrical info
    duration = na["duration_div"]/na["tot_measure_div"]
    # TODO: consider rests as lag from previous note
    return np.concatenate((pitch, metrical, duration), axis=1)


In [ ]:
na

array([( 0. , 1. ,  0.  , 0.5 ,   0,  2, 70, 1, 'None',  1, 8,  1, 1, 22,  2, 4),
       ( 1. , 4. ,  0.5 , 2.  ,   2,  8, 79, 1, 'None', 12, 8,  4, 1,  0, 24, 4),
       ( 5. , 1. ,  2.5 , 0.5 ,  10,  2, 77, 1, 'None', 12, 8,  4, 0,  8, 24, 4),
       ( 6. , 1. ,  3.  , 0.5 ,  12,  2, 79, 1, 'None', 12, 8,  4, 0, 10, 24, 4),
       ( 7. , 3. ,  3.5 , 1.5 ,  14,  6, 77, 1, 'None', 12, 8,  4, 0, 12, 24, 4),
       (10. , 2. ,  5.  , 1.  ,  20,  4, 75, 1, 'None', 12, 8,  4, 0, 18, 24, 4),
       (12. , 1. ,  6.  , 0.5 ,  24,  2, 70, 1, 'None', 12, 8,  4, 0, 22, 24, 4),
       (13. , 2. ,  6.5 , 1.  ,  26,  4, 79, 1, 'None', 12, 8,  4, 1,  0, 24, 4),
       (15. , 1. ,  7.5 , 0.5 ,  30,  2, 72, 1, 'None', 12, 8,  4, 0,  4, 24, 4),
       (16. , 2. ,  8.  , 1.  ,  32,  4, 84, 1, 'None', 12, 8,  4, 0,  6, 24, 4),
       (18. , 1. ,  9.  , 0.5 ,  36,  2, 79, 1, 'None', 12, 8,  4, 0, 10, 24, 4),
       (19. , 3. ,  9.5 , 1.5 ,  38,  6, 82, 1, 'None', 12, 8,  4, 0, 12, 24, 4),
       (22. , 2.

# Test data loader

In [ ]:
from musicparser.data_loading import TSDataset
import partitura as pt
from pathlib import Path

c:\Users\fosca\.conda\envs\mir_research\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset = TSDataset(Path("data"))

!!!!! Error with 01_Waltz in E flat Grande Valse Brillante Op.18.txt expected np.ndarray (got list)
!!!!! Error with 02_Moments Musicaux.txt expected np.ndarray (got list)
!!!!! Error with 03_Bagatelle 'Fur Elise' WoO.59.txt expected np.ndarray (got list)
!!!!! Error with 04_The Preludes Op.28 No.15.txt expected np.ndarray (got list)
!!!!! Error with 05_Turkish March.txt expected np.ndarray (got list)
!!!!! Error with 06_Blumenlied Op.39.txt expected np.ndarray (got list)
!!!!! Error with 07_Nocturne.txt expected np.ndarray (got list)
!!!!! Error with 08_Spinnerlied Op.14 No.4.txt expected np.ndarray (got list)
!!!!! Error with 09_String Quartet in F major Op.3 No.5 Serenade.txt expected np.ndarray (got list)
!!!!! Error with 10_Wiegenlied.txt expected np.ndarray (got list)
!!!!! Error with 100_Sonate fur Klavier Nr.11 A dur K.331 K6.300i Mov.1.txt expected np.ndarray (got list)
!!!!! Error with 101_unknown.txt expected np.ndarray (got list)
!!!!! Error with 102_unknown.txt expected np

In [ ]:
# a = np.zeros((2,5))
for e in dataset:
    print(e)
    break

(array([[70.        ,  1.        ,  0.33333333],
       [74.        ,  0.        ,  0.16666667],
       [75.        ,  0.        ,  0.16666667],
       [77.        ,  0.        ,  0.33333333],
       [70.        ,  1.        ,  0.33333333],
       [75.        ,  0.        ,  0.16666667],
       [77.        ,  0.        ,  0.16666667],
       [79.        ,  0.        ,  0.33333333],
       [70.        ,  1.        ,  0.33333333],
       [77.        ,  0.        ,  0.16666667],
       [79.        ,  0.        ,  0.16666667],
       [80.        ,  0.        ,  0.33333333],
       [82.        ,  1.        ,  0.66666667],
       [82.        ,  0.        ,  0.16666667],
       [82.        ,  0.        ,  0.16666667],
       [82.        ,  1.        ,  0.33333333],
       [84.        ,  0.        ,  0.16666667],
       [82.        ,  0.        ,  0.16666667],
       [80.        ,  0.        ,  0.33333333],
       [80.        ,  1.        ,  0.33333333],
       [82.        ,  0.        ,  0.16

In [ ]:
# score = pt.load_musicxml(r"data\127\MSC-127.xml") 
# score = pt.load_musicxml(r"data\07\07_Nocturne.xml")
onsets = score.parts[0].note_array()["onset_div"]

c:\Users\fosca\.conda\envs\mir_research\lib\site-packages\partitura\directions.py:533: UserWarning: error parsing "cre    -     -     -     scen     -    -    -     do" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))


In [ ]:
from musicparser.data_loading import TSDataset
import partitura as pt
from pathlib import Path
from musicparser.data_loading import get_dependency_arcs, get_note_features_and_dep_arcs
# score = pt.load_musicxml(Path(r"data/gttm/283/score.xml"), force_note_ids=True)

# arcs, gttm_arcs =get_dependency_arcs(Path(r"data/283/TS.xml"),score)
get_note_features_and_dep_arcs("data/gttm/291/score.xml","data/gttm/291/TS.xml")

Trying to solve first measure error in:  data/gttm/291/TS.xml


ValueError: Can't assign ids in: data/gttm/291/TS.xml, error: Trying to build an arc from a rest

In [ ]:
for e in zip(arcs,gttm_arcs):
    print(e)

((38, 39), ('P1-11-5', 'P1-11-6'))
((40, 41), ('P1-11-7', 'P1-11-8'))
((38, 40), ('P1-11-5', 'P1-11-7'))
((36, 35), ('P1-11-3', 'P1-11-2'))
((36, 37), ('P1-11-3', 'P1-11-4'))
((38, 36), ('P1-11-5', 'P1-11-3'))
((42, 38), ('P1-12-1', 'P1-11-5'))
((32, 33), ('P1-10-2', 'P1-10-3'))
((34, 32), ('P1-10-4', 'P1-10-2'))
((31, 30), ('P1-10-1', 'P1-9-4'))
((29, 28), ('P1-9-3', 'P1-9-2'))
((31, 29), ('P1-10-1', 'P1-9-3'))
((34, 31), ('P1-10-4', 'P1-10-1'))
((42, 34), ('P1-12-1', 'P1-10-4'))
((1, 0), ('P1-2-1', 'P1-1-2'))
((2, 3), ('P1-2-2', 'P1-2-3'))
((4, 2), ('P1-2-4', 'P1-2-2'))
((1, 4), ('P1-2-1', 'P1-2-4'))
((8, 9), ('P1-3-5', 'P1-3-6'))
((10, 11), ('P1-3-7', 'P1-3-8'))
((8, 10), ('P1-3-5', 'P1-3-7'))
((6, 5), ('P1-3-3', 'P1-3-2'))
((6, 7), ('P1-3-3', 'P1-3-4'))
((8, 6), ('P1-3-5', 'P1-3-3'))
((12, 8), ('P1-4-1', 'P1-3-5'))
((1, 12), ('P1-2-1', 'P1-4-1'))
((23, 24), ('P1-7-5', 'P1-7-6'))
((25, 26), ('P1-7-7', 'P1-7-8'))
((23, 25), ('P1-7-5', 'P1-7-7'))
((21, 20), ('P1-7-3', 'P1-7-2'))
((21,

In [ ]:
import torch
a = torch.tensor([[1,2,3],[4,5,6],[7,8,9]])
a_indices = torch.arange(len(a))
cart_prod = torch.cartesian_prod(a_indices,a_indices)
# remove self loops
cart_prod = cart_prod[cart_prod[:,0]!=cart_prod[:,1]]
# concatenate
out = torch.cat([a[cart_prod[:,0]],a[cart_prod[:,1]]],dim=-1)
out.view(-1)

tensor([1, 2, 3, 4, 5, 6, 1, 2, 3, 7, 8, 9, 4, 5, 6, 1, 2, 3, 4, 5, 6, 7, 8, 9,
        7, 8, 9, 1, 2, 3, 7, 8, 9, 4, 5, 6])

In [ ]:
import partitura as pt

/share/home/francesco/miniconda3/envs/musicparser/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
score = pt.load_musicxml(Path(r"data\288\score.xml"), force_note_ids=True)
for e in score.parts[0].iter_all(pt.score.ge)

SyntaxError: invalid syntax (3043171944.py, line 2)

# GET number of total durations

In [ ]:
from musicparser.data_loading import TSDataset
from pathlib import Path
import warnings
warnings.filterwarnings('ignore') # setting ignore as a parameter
import torch
import numpy as np

/share/home/francesco/miniconda3/envs/musicparser/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset = TSDataset(Path("data"), data_augmentation= True)

Loading data...
!!!!! Error with 154_unknown.txt 
Trying to solve first measure error in:  data/276/TS.xml
Trying to solve first measure error in:  data/278/TS.xml
Trying to solve first measure error in:  data/280/TS.xml
Trying to solve first measure error in:  data/283/TS.xml
!!!!! Error with 283_unknown.txt ("Can't assign ids in: ", 'data/283/TS.xml')
Trying to solve first measure error in:  data/285/TS.xml
Trying to solve first measure error in:  data/288/TS.xml
!!!!! Error with 288_unknown.txt ("Can't assign ids in: ", 'data/288/TS.xml')
Trying to solve first measure error in:  data/291/TS.xml
!!!!! Error with 291_unknown.txt ("Can't assign ids in: ", 'data/291/TS.xml')


In [ ]:
for e in dataset.note_features:
    print(e[:,2])
    break

tensor([0.3333, 0.1667, 0.1667, 0.3333, 0.3333, 0.1667, 0.1667, 0.3333, 0.3333,
        0.1667, 0.1667, 0.3333, 0.6667, 0.1667, 0.1667, 0.3333, 0.1667, 0.1667,
        0.3333, 0.3333, 0.1667, 0.1667, 0.3333, 0.3333, 0.1667, 0.1667, 0.3333,
        1.0000], dtype=torch.float64)


In [ ]:
torch.Tensor(dataset.note_features)

ValueError: only one element tensors can be converted to Python scalars

In [ ]:
torch.cat(dataset.note_features, dim=0)[:,2]

tensor([0.3333, 0.1667, 0.1667,  ..., 0.0833, 0.0833, 0.3333],
       dtype=torch.float64)

In [ ]:
torch.unique(torch.cat(dataset.note_features, dim=0)[:,2])

tensor([0.0312, 0.0357, 0.0417, 0.0500, 0.0556, 0.0625, 0.0833, 0.1111, 0.1250,
        0.1667, 0.1750, 0.1875, 0.2083, 0.2222, 0.2500, 0.2917, 0.3125, 0.3333,
        0.3750, 0.4062, 0.4167, 0.4375, 0.4444, 0.5000, 0.5556, 0.5625, 0.5833,
        0.6250, 0.6667, 0.6875, 0.7500, 0.8333, 0.8750, 0.9167, 0.9375, 1.0000,
        1.1250, 1.1667, 1.2500, 1.3333, 1.5000, 1.6667, 1.7500, 2.0000, 4.0000],
       dtype=torch.float64)

In [ ]:
for i,e in enumerate(dataset.note_features):
    # if torch.any(e[:,2] == 0):
    print(dataset.score_files[i])

data/01/score.xml
data/02/score.xml
data/03/score.xml
data/04/score.xml
data/05/score.xml
data/06/score.xml
data/07/score.xml
data/08/score.xml
data/09/score.xml
data/10/score.xml
data/100/score.xml
data/101/score.xml
data/102/score.xml
data/103/score.xml
data/104/score.xml
data/105/score.xml
data/106/score.xml
data/107/score.xml
data/108/score.xml
data/109/score.xml
data/11/score.xml
data/110/score.xml
data/111/score.xml
data/112/score.xml
data/113/score.xml
data/114/score.xml
data/115/score.xml
data/116/score.xml
data/117/score.xml
data/118/score.xml
data/119/score.xml
data/12/score.xml
data/120/score.xml
data/121/score.xml
data/122/score.xml
data/123/score.xml
data/124/score.xml
data/125/score.xml
data/126/score.xml
data/127/score.xml
data/128/score.xml
data/129/score.xml
data/13/score.xml
data/130/score.xml
data/131/score.xml
data/132/score.xml
data/133/score.xml
data/134/score.xml
data/135/score.xml
data/136/score.xml
data/137/score.xml
data/138/score.xml
data/139/score.xml
data/1

In [ ]:
from musicparser.data_loading import get_nra
import partitura as pt
score =pt.load_musicxml(r"/share/home/francesco/musicparser/data/283/score.xml")
nra = get_nra(score)

In [ ]:
for note in score.parts[0].notes:
    print(note)

20--24 Note id=None voice=1 staff=1 type=eighth pitch=B4
24--36 Note id=None voice=1 staff=1 type=quarter. articulations=(accent) pitch=E5
36--40 Note id=None voice=1 staff=1 type=eighth articulations=(accent) pitch=E5
40--44 Note id=None voice=1 staff=1 type=eighth articulations=(accent) pitch=E5
44--48 Note id=None voice=1 staff=1 type=eighth articulations=(accent) pitch=E5
48--60 Note id=None voice=1 staff=1 type=quarter. articulations=(accent) pitch=D#5
60--64 Note id=None voice=1 staff=1 type=eighth articulations=(accent) pitch=D#5
64--68 Note id=None voice=1 staff=1 type=eighth articulations=(accent) pitch=D#5
68--72 Note id=None voice=1 staff=1 type=eighth articulations=(accent) pitch=F#5
72--96 Note id=None voice=1 staff=1 type=half. tie_group=None+None pitch=C#5
72--72 GraceNote id=None voice=1 staff=1 type=eighth articulations=(accent) pitch=F#5 main_note=72--96 Note id=None voice=1 staff=1 type=half. tie_group=None+None pitch=C#5
96--100 Note id=None voice=1 staff=1 type=eig

# Load jazz data

In [ ]:
import json
from pathlib import Path
import numpy as np

In [ ]:
with open(str(Path("data/jazz_tb/treebank.json"))) as f:
    all_data = json.load(f)
# remove pieces with no tree annotations
data = [e for e in all_data if e.get("trees") is not None]

In [ ]:
meters = np.unique([e["meter"]["numerator"] for e in all_data])
print(meters)
for piece in all_data:
    if piece["meter"]["numerator"] == 6:
        print(piece["title"], piece["meter"])
        for b, c in zip(piece["beats"],piece["chords"]):
            print(b,c)

[3 4 6]
Wild Flower {'numerator': 6, 'denominator': 4}
1 Bb^7
1 Abm7
4 A7
1 Dsus
1 Dsus
1 Bb^7
1 Abm7
4 A7
1 D7
1 D7
1 Gsus
1 Cm7
4 F7
1 Bb^7
1 Ab^7
1 Gb7
1 C7
1 Fm
1 E7
1 Eb^7
4 Cm7
1 Abm7
4 A7
1 Dsus
1 Dsus
1 Bb^7
1 Abm7
4 A7
1 D7
1 D7
1 Gsus
1 Cm7
3 F7
1 Bb^7
3 Bbm7
1 Bbm7
3 Eb7
1 Ab^7
3 Db^7
1 Abm7
3 Eb7
1 D7
1 D7
West Coast Blues {'numerator': 6, 'denominator': 4}
1 Bb7
1 Ab7
1 Bb7
1 Bm7
4 E7
1 Eb7
1 Eb7
1 Bb7
1 Bb7
1 F7
1 Eb7
1 Bb7
1 Bb7
1 Bb7
1 Ab7
1 Bb7
1 Bm7
4 E7
1 Eb7
1 Eb7
1 Bb7
1 Bb7
1 F7
1 Eb7
1 Bb7
1 Bb7
1 Bb7
1 Ab7
1 Bb7
1 Bm7
4 E7
1 Eb7
1 Ebm7
4 Ab7
1 Dm7
4 G7
1 C#m7
4 F#7
1 Cm7
1 Cm7
4 F7
1 Bb^7
4 Db7
1 Gb^7
4 F7
Benny's Tune {'numerator': 6, 'denominator': 4}
1 Gbm6
1 Db^
1 Gbm6
1 Db^
1 Gbm6
1 Db^
1 Gbm6
1 Db^
1 B^7
1 Bbm7
1 F^
1 Absus
1 Eb^
1 Gb7
1 Db^
1 B7
1 G%7
1 Gbm6
1 Db^
1 B^7
1 A^7
1 D^
1 Db^
1 Gbm6
1 Db^
1 Db^
1 Gbm6
1 Db^
1 Gbm6
1 Db^
1 Gbm6
1 Db^
1 Gbm6
1 Db^
1 Gbm6
1 Db^
1 B^7
1 Bbm7
1 F^
1 Absus
1 Eb^
1 Gb7
1 Db^
1 B7
1 G%7
1 Gbm6
1 Db^
1 B^7
1 A^7
1 D^
1 

In [ ]:
pieces34 = [e for e in data if e["meter"]["numerator"] == 3]
for piece in pieces34:
    print(piece["title"],piece["beats"])

Booker's Waltz [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Contemplation [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Valse Hot [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
We Will Meet Again [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
United [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1]
Funkallero [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 1, 1, 1]
Afro Blue [1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1]
Bluesette [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Footprints [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Simone [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Night Dreamer [1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 3]
Gary's Notebook [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Someday My Prince Will Come [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
for i, e in enumerate(data):
    print(i,e["title"], e["meter"])

0 Red Clay {'numerator': 4, 'denominator': 4}
1 Equinox {'numerator': 4, 'denominator': 4}
2 Jody Grind, The {'numerator': 4, 'denominator': 4}
3 Mr. P.C. {'numerator': 4, 'denominator': 4}
4 Blues For Wood {'numerator': 4, 'denominator': 4}
5 Interplay {'numerator': 4, 'denominator': 4}
6 Solar {'numerator': 4, 'denominator': 4}
7 Birk's Works {'numerator': 4, 'denominator': 4}
8 Blue In Green {'numerator': 4, 'denominator': 4}
9 Booker's Waltz {'numerator': 3, 'denominator': 4}
10 Contemplation {'numerator': 3, 'denominator': 4}
11 Friday The 13th {'numerator': 4, 'denominator': 4}
12 If You Never Come To Me {'numerator': 4, 'denominator': 4}
13 Pent Up House {'numerator': 4, 'denominator': 4}
14 Sweet Georgia Bright {'numerator': 4, 'denominator': 4}
15 Valse Hot {'numerator': 3, 'denominator': 4}
16 Wayne's Thang {'numerator': 4, 'denominator': 4}
17 We Will Meet Again {'numerator': 3, 'denominator': 4}
18 Blue Bossa {'numerator': 4, 'denominator': 4}
19 Israel {'numerator': 4, 'de

In [ ]:
piece_index = 18
piece = data[piece_index]
for beat, chord in zip(piece["beats"],piece["chords"]):
    print(beat, chord)

1 Cm7
1 Cm7
1 Fm7
1 Fm7
1 D%7
1 G7
1 Cm7
1 Cm7
1 Ebm7
1 Ab7
1 Db^7
1 Db^7
1 D%7
1 G7
1 Cm7
1 D%7
3 G7


In [ ]:
tree = piece["trees"][0]["open_constituent_tree"]
tree

{'label': 'Cm7',
 'children': [{'label': 'Cm7',
   'children': [{'label': 'Cm7',
     'children': [{'label': 'Cm7', 'children': []},
      {'label': 'Cm7', 'children': []}]},
    {'label': 'Cm7',
     'children': [{'label': 'G7',
       'children': [{'label': 'Fm7',
         'children': [{'label': 'Fm7', 'children': []},
          {'label': 'Fm7', 'children': []}]},
        {'label': 'G7',
         'children': [{'label': 'D%7', 'children': []},
          {'label': 'G7', 'children': []}]}]},
      {'label': 'Cm7',
       'children': [{'label': 'Cm7', 'children': []},
        {'label': 'Cm7', 'children': []}]}]}]},
  {'label': 'Cm7',
   'children': [{'label': 'Db^7',
     'children': [{'label': 'Ab7',
       'children': [{'label': 'Ebm7', 'children': []},
        {'label': 'Ab7', 'children': []}]},
      {'label': 'Db^7',
       'children': [{'label': 'Db^7', 'children': []},
        {'label': 'Db^7', 'children': []}]}]},
    {'label': 'Cm7',
     'children': [{'label': 'G7',
       'chi

In [ ]:
def parse_jht_to_dep_tree(jht_dict):
    """Parse the python jazz harmony tree dict to a list of dependencies and a list of chord in the leaves.
    """
    all_leaves = []
    all_indices = []

    def _iterative_parse_jht(dict_elem):
        """Iterative function to parse the python jazz harmony tree dict to a list of dependencies."""
        children = dict_elem["children"]
        if children == []:  # recursion ending condition
            out = (
                [],
                {"index": len(all_leaves), "label": dict_elem["label"]},
            )
            # add the label of the current node to the global list of leaves
            all_indices.append(len(all_leaves))
            all_leaves.append(dict_elem["label"])
            return out
        else:  # recursive call
            assert len(children) == 2 
            current_label = dict_elem["label"]
            out_list = []  # dependency list
            iterative_result_left = _iterative_parse_jht(children[0])
            iterative_result_right = _iterative_parse_jht(children[1])
            # merge the dependencies lists computed deeper
            out_list.extend(iterative_result_left[0])
            out_list.extend(iterative_result_right[0])
            # check if the label correspond to the left or right children and return the corresponding result
            if iterative_result_right[1]["label"] == current_label: # default if both children are equal is to go left-right arch
                # append the dependency for the current node
                out_list.append((iterative_result_right[1]["index"], iterative_result_left[1]["index"]))
                return out_list, iterative_result_right[1]
            elif iterative_result_left[1]["label"] == current_label: 
                print("right-left arc on label", current_label)
                # append the dependency for the current node
                out_list.append((iterative_result_left[1]["index"], iterative_result_right[1]["index"]))
                return out_list, iterative_result_left[1]
            else:
                raise ValueError("Something went wrong with label", current_label)
            
    dep_arcs, root = _iterative_parse_jht(jht_dict)
    return dep_arcs, all_leaves



In [ ]:
for piece in data:
    print(piece["title"],"-------------------")
    d_arcs, leaves = parse_jht_to_dep_tree(piece["trees"][0]["complete_constituent_tree"])
    if piece.get("turnaround") is None:
        piece["turnaround"] = 0
    if piece["turnaround"] == 0:
        chords = piece["chords"]
    elif piece["turnaround"] > 0:
        chords = piece["chords"][:-piece["turnaround"]]
    elif piece["turnaround"] == -1:
        chords = piece["chords"] + [piece["chords"][0]]
    else:
        raise ValueError("Turnaround value not recognized")
    # if leaves != chords:
    #     print(piece["title"],"-------------------")
    #     print(leaves)
    #     print(chords)

Red Clay -------------------
Equinox -------------------
Jody Grind, The -------------------
Mr. P.C. -------------------
Blues For Wood -------------------
Interplay -------------------
Solar -------------------
Birk's Works -------------------
Blue In Green -------------------
Booker's Waltz -------------------
Contemplation -------------------
Friday The 13th -------------------
If You Never Come To Me -------------------
Pent Up House -------------------
Sweet Georgia Bright -------------------
Valse Hot -------------------
Wayne's Thang -------------------
We Will Meet Again -------------------
Blue Bossa -------------------
Israel -------------------
right-left arc on label Dm
right-left arc on label Dm
Mac The Knife -------------------
Sugar -------------------
When The Saints Go Marching In -------------------
right-left arc on label Bb^
Work Song -------------------
Beatrice -------------------
right-left arc on label F^7
Jackie-ing -------------------
Lady Bird --------------

In [ ]:
for piece in data:
    d_arcs, leaves = parse_jht_to_dep_tree(piece["trees"][0]["complete_constituent_tree"])
    if piece.get("turnaround") is None:
        piece["turnaround"] = 0
    if piece["turnaround"] == 0:
        chords = piece["chords"]
    elif piece["turnaround"] > 0:
        chords = piece["chords"][:-piece["turnaround"]]
    elif piece["turnaround"] == -1:
        chords = piece["chords"] + [piece["chords"][0]]
    else:
        raise ValueError("Turnaround value not recognized")
    if leaves != chords:
        print(piece["title"],"-------------------")
        print(leaves)
        print(chords)

Interplay -------------------
['Fm6', 'Fm6', 'Fm6', 'Fm6', 'F7', 'Bbm7', 'Bbm7', 'Fm6', 'Fm6', 'Db7', 'C7', 'Fm6']
['Fm6', 'Fm6', 'Fm6', 'Fm6', 'F7', 'Bbm7', 'Bbm7', 'Fm6', 'Fm6', 'Db7', 'C7', 'Fm6', 'G%7', 'C7']
Solar -------------------
['Cm^7', 'Cm^7', 'Gm7', 'C7', 'F^7', 'F^7', 'Fm7', 'Bb7', 'Eb^7', 'Ebm7', 'Ab7', 'Db^7', 'D%7', 'G7', 'Cm^7']
['Cm^7', 'Cm^7', 'Gm7', 'C7', 'F^7', 'F^7', 'Fm7', 'Bb7', 'Eb^7', 'Ebm7', 'Ab7', 'Db^7', 'D%7', 'G7']
Friday The 13th -------------------
['G^7', 'F7', 'Eb7', 'D7', 'G^7', 'F7', 'Eb7', 'D7', 'G^7', 'F7', 'Eb7', 'D7', 'G^7', 'F7', 'Eb7', 'D7', 'G^7']
['G^7', 'F7', 'Eb7', 'D7', 'G^7', 'F7', 'Eb7', 'D7', 'G^7', 'F7', 'Eb7', 'D7', 'G^7', 'F7', 'Eb7', 'D7']
right-left arc on label Dm
right-left arc on label Dm
right-left arc on label Bb^
right-left arc on label F^7
right-left arc on label A^7
right-left arc on label Dm
right-left arc on label Dm
right-left arc on label Am
right-left arc on label F^
right-left arc on label Cm7
right-left arc on labe

In [ ]:
data[5]["turnaround"]
print(data[5]["chords"])
data[5]["trees"][0]["complete_constituent_tree"]


['Fm6', 'Fm6', 'Fm6', 'Fm6', 'F7', 'Bbm7', 'Bbm7', 'Fm6', 'Fm6', 'Db7', 'C7', 'Fm6', 'G%7', 'C7']


{'label': 'Fm6',
 'children': [{'label': 'Fm6',
   'children': [{'label': 'Fm6',
     'children': [{'label': 'Fm6',
       'children': [{'label': 'Fm6', 'children': []},
        {'label': 'Fm6', 'children': []}]},
      {'label': 'Fm6',
       'children': [{'label': 'Fm6', 'children': []},
        {'label': 'Fm6', 'children': []}]}]},
    {'label': 'Fm6',
     'children': [{'label': 'Bbm7',
       'children': [{'label': 'F7', 'children': []},
        {'label': 'Bbm7',
         'children': [{'label': 'Bbm7', 'children': []},
          {'label': 'Bbm7', 'children': []}]}]},
      {'label': 'Fm6',
       'children': [{'label': 'Fm6', 'children': []},
        {'label': 'Fm6', 'children': []}]}]}]},
  {'label': 'Fm6',
   'children': [{'label': 'C7',
     'children': [{'label': 'Db7', 'children': []},
      {'label': 'C7', 'children': []}]},
    {'label': 'Fm6', 'children': []}]}]}

In [ ]:
def get_features_from_chord_label(label):
    if label[1] ==  "#" or label[1] == "b":
        root = label[:2]
        cf_index = 2
    else:
        root = label[0]
        cf_index = 1
    chord_form = label[cf_index]

In [ ]:
import re
p = re.compile('[m+]|sus')

m = p.match("Cm7")
m

In [ ]:
def parse_chord_label(chord_label):
  # Define a regex pattern for chord symbols
  pattern = r"([A-G][#b]?)(m|\+|%|o|sus)?(6|7|\^7)?"
  # Match the pattern with the input chord
  match = re.match(pattern, chord_label)
  if match:
    # Extract the root, basic chord form and optional added note from the match object
    root = match.group(1)
    form = match.group(2) or "M"
    note = match.group(3) or ""
    return root, form, note
  else:
    # Return None if the input is not a valid chord symbol
    raise ValueError("Invalid chord symbol: {}".format(chord_label))
  
parse_chord_label("Gm7")

('G', 'm', '7')

In [ ]:
all_durations = []
for piece in all_data:
    print(piece["title"], piece["meter"])
    total_beats = piece["meter"]["numerator"]
    for b,c in zip(piece["beats"],piece["chords"]):
        print(total_beats, b, c, (total_beats - b +1) / total_beats)
        all_durations.append((total_beats - b +1) / total_beats)



Sweeping Up {'numerator': 3, 'denominator': 4}
3 1 D7 1.0
3 1 G^7 1.0
3 1 F#m7 1.0
3 1 Bm 1.0
3 1 Em7 1.0
3 1 Bm 1.0
3 1 Am7 1.0
Red Clay {'numerator': 4, 'denominator': 4}
4 1 Cm7 1.0
4 1 Bbm7 1.0
4 1 Dbsus 1.0
4 3 Ebsus 0.5
4 1 Fsus 1.0
4 3 Gsus 0.5
4 1 Cm7 1.0
4 1 Bbm7 1.0
4 3 Eb7 0.5
4 1 Ab^7 1.0
4 1 D%7 1.0
4 3 G7 0.5
Bessie's Blues {'numerator': 4, 'denominator': 4}
4 1 Eb7 1.0
4 1 Ab7 1.0
4 1 Eb7 1.0
4 1 Eb7 1.0
4 1 Ab7 1.0
4 1 Ab7 1.0
4 1 Eb7 1.0
4 1 Eb7 1.0
4 1 Bb7 1.0
4 1 Ab7 1.0
4 1 Eb7 1.0
4 1 Bb7 1.0
Blues Five Spot {'numerator': 4, 'denominator': 4}
4 1 Bb7 1.0
4 1 Eb7 1.0
4 1 Bb7 1.0
4 1 Bb7 1.0
4 1 Eb7 1.0
4 1 Ab7 1.0
4 1 Bb7 1.0
4 1 Bb7 1.0
4 1 F7 1.0
4 1 F7 1.0
4 1 Bb7 1.0
4 1 Bb7 1.0
Blues In The Closet {'numerator': 4, 'denominator': 4}
4 1 Ab7 1.0
4 1 Ab7 1.0
4 1 Ab7 1.0
4 1 Ab7 1.0
4 1 Db7 1.0
4 1 Db7 1.0
4 1 Ab7 1.0
4 1 Ab7 1.0
4 1 Bbm7 1.0
4 1 Eb7 1.0
4 1 Ab7 1.0
4 1 Ab7 1.0
Blue Sphere {'numerator': 4, 'denominator': 4}
4 1 Bb7 1.0
4 1 Eb7 1.0
4 1 Bb7 1.0
4 1 B

(array([-1.        , -0.75      , -0.5       , -0.25      ,  0.        ,
         0.25      ,  0.33333333,  0.5       ,  0.66666667,  0.75      ,
         1.        ]),
 array([    1,     1,    34,     2,   112,   837,   221, 15216,    11,
          408, 42307]))

In [ ]:
np.unique(all_durations, return_counts=True)

(array([-1.        , -0.75      , -0.5       , -0.25      ,  0.        ,
         0.25      ,  0.33333333,  0.5       ,  0.66666667,  0.75      ,
         1.        ]),
 array([    1,     1,    34,     2,   112,   837,   221, 15216,    11,
          408, 42307]))

In [ ]:
all_durations = []
for piece in data:
    # print(piece["title"], piece["meter"])
    total_beats = piece["meter"]["numerator"]
    for b,c in zip(piece["beats"],piece["chords"]):
        # print(total_beats, b, c, (total_beats - b +1) / total_beats)
        all_durations.append((total_beats - b +1) / total_beats)
np.unique(all_durations, return_counts=True)

(array([0.25      , 0.33333333, 0.5       , 0.75      , 1.        ]),
 array([  25,   26,  511,   11, 3589]))

In [ ]:
all_roots = []
all_chord_type = []
all_extension = []
for piece in all_data:
    # print(piece["title"], piece["meter"])
    for chord in piece["chords"]:
        root, chord_type, extension = parse_chord_label(chord)
        all_roots.append(root)
        all_chord_type.append(chord_type)
        all_extension.append(extension)

print(np.unique(all_roots))
print(np.unique(all_chord_type))
print(np.unique(all_extension))

['A' 'A#' 'Ab' 'B' 'Bb' 'C' 'C#' 'Cb' 'D' 'D#' 'Db' 'E' 'Eb' 'F' 'F#' 'G'
 'G#' 'Gb']
['%' '+' 'M' 'm' 'o' 'sus']
['' '6' '7' '^7']


ModuleNotFoundError: No module named 'music21'

# Check for problems in the d_tree to c_tree conversion

In [ ]:
from musicparser.data_loading import JTBDataset, TSDataset
import numpy as np
from collections import defaultdict
import re

def identify_conversion_conflicts(d_arcs):
    # find if exist one node that have one right dependant and one left dependant
    d = defaultdict(list)
    for arc in d_arcs:
        d[arc[0]].append(arc[1])
    # now check if inside each list there is are elements that are both smaller and bigger than the key
    for k, v in d.items():
        if len(v) > 1:
            if min(v) < k < max(v):
                return True  


/share/home/francesco/miniconda3/envs/musicparser/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# check TS Dataset
dataset = TSDataset("data/gttm/", n_jobs=1)

for i,e in enumerate(dataset):
    title = dataset.score_files[i]
    d_arcs = e[2][e[1]]
    if identify_conversion_conflicts(d_arcs):
        print(title)
        print(d_arcs)


Loading data...


  0%|          | 0/300 [00:00<?, ?it/s]/share/home/francesco/musicparser/musicparser/data_loading.py:225: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525551200/work/aten/src/ATen/native/IndexingUtils.h:27.)
  pot_arcs = pot_arcs[~np.logical_or(starting_rest_mask, ending_rest_mask)]
  9%|▊         | 26/300 [00:01<00:12, 22.60it/s]/share/home/francesco/miniconda3/envs/musicparser/lib/python3.9/site-packages/partitura/directions.py:533: UserWarning: error parsing "sentito" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
/share/home/francesco/miniconda3/envs/musicparser/lib/python3.9/site-packages/partitura/directions.py:533: UserWarning: error parsing "Largamente" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
 20%|█▉        | 59/300 [00:02<00:11, 20.21it/s]/share/home/fra

!!!!! Error with 154_unknown.txt 


 26%|██▌       | 78/300 [00:03<00:12, 18.22it/s]/share/home/francesco/miniconda3/envs/musicparser/lib/python3.9/site-packages/partitura/io/importmusicxml.py:871: UserWarning: ignoring direction type: octave-shift {'default-y': '33', 'size': '8', 'type': 'down'}
  warnings.warn("ignoring direction type: {} {}".format(dt.tag, dt.attrib))
/share/home/francesco/miniconda3/envs/musicparser/lib/python3.9/site-packages/partitura/io/importmusicxml.py:871: UserWarning: ignoring direction type: octave-shift {'size': '8', 'type': 'stop'}
  warnings.warn("ignoring direction type: {} {}".format(dt.tag, dt.attrib))
 30%|███       | 90/300 [00:04<00:09, 23.33it/s]

Trying to solve first measure error in:  data/gttm/169/TS.xml
!!!!! Error with 169_unknown.txt Can't assign ids in: data/gttm/169/TS.xml, error: index 3 is out of bounds for axis 0 with size 3


...

 75%|███████▌  | 226/300 [00:10<00:02, 31.77it/s]

Trying to solve first measure error in:  data/gttm/291/TS.xml
!!!!! Error with 291_unknown.txt Can't assign ids in: data/gttm/291/TS.xml, error: Trying to build an arc from a rest


 96%|█████████▌| 287/300 [00:12<00:00, 26.77it/s]/share/home/francesco/miniconda3/envs/musicparser/lib/python3.9/site-packages/partitura/io/importmusicxml.py:871: UserWarning: ignoring direction type: octave-shift {'default-y': '22', 'size': '8', 'type': 'down'}
  warnings.warn("ignoring direction type: {} {}".format(dt.tag, dt.attrib))
100%|██████████| 300/300 [00:12<00:00, 23.60it/s]


Removed 4 scores due to errors


/share/home/francesco/musicparser/musicparser/data_loading.py:572: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  n_feats = torch.tensor(n_feats)


In [ ]:
# check Jazz Dataset
dataset = JTBDataset("data/jazz_tb/treebank.json", data_augmentation="no", only_tree=True)

for i,e in enumerate(dataset):
    title = dataset.titles[i]
    d_arcs = e[2][e[1]]
    if identify_conversion_conflicts(d_arcs):
        print(title)
        print(d_arcs)

Loading data...
Done loading data. 0 out of 150 pieces were discarded because of errors.


# Transform from d-tree to c-tree

In [62]:
import numpy as np
from musicparser.data_loading import JTBDataset, TSDataset
from collections import defaultdict
import re

In [63]:
class Node:
    def __init__(self,label):
        self.label = label
        self.children = []

    def add_child(self, node):
        self.children.append(node)

    def set_children(self, node1, node2):
        self.children = [node1, node2]

    def unlabeled_repr(self):
        if len(self.children) == 0:
            return self.label
        else:
            return [self.children[0].unlabeled_repr(),self.children[1].unlabeled_repr()]

    def __str__(self):
        return f"N{self.label}"


In [64]:
def dtree2unlabeled_ctree(d_arcs):
    # build head dictionary
    d = defaultdict(list)
    for arc in d_arcs:
        d[arc[0]].append(arc[1])
    # ensure the lists in the dictionary are sorted
    for v in d.values():
        v.sort()
    # find root
    head_noted = [arc[0] for arc in d_arcs]
    dependent_nodes = [arc[1] for arc in d_arcs]
    potential_roots = np.unique([k for k in head_noted if k not in dependent_nodes])
    assert(len(potential_roots) == 1)
    root = potential_roots[0]
    # build ctree
    node_root = Node(root)

    def _recursive_topdown_build(d, node):
        # stopping condition
        if node.label not in d or len(d[node.label]) == 0:
            return
        else:
        # the indices of dependent will be all smaller or all bigger than root, so we can check the first one
            if d[node.label][0] > root: # dependent on the right
                left_node_child = Node(node.label)
                right_node_child = Node(d[node.label][-1])
                # remove the value from the list
                d[node.label].pop()
            else:
                left_node_child = Node(d[node.label][0])
                right_node_child = Node(node.label)
                # remove the value from the list
                d[node.label].pop(0)
            # set childrens and iterate on them
            node.set_children(left_node_child, right_node_child)
            _recursive_topdown_build(d, left_node_child)
            _recursive_topdown_build(d, right_node_child)
    
        
    _recursive_topdown_build(d, node_root)
    return node_root
    

dep_test = [(0,1),(1,2),(1,3),(3,4)]
c_tree = dtree2unlabeled_ctree(dep_test)
c_tree.unlabeled_repr()

[0, [[1, 2], [3, 4]]]

In [68]:
from musicparser.data_loading import JTBDataset, TSDataset

dataset = JTBDataset("data/jazz_tb/treebank.json", data_augmentation="no", only_tree=True)

for i,e in enumerate(dataset):
    title = dataset.titles[i]
    if title == "Booker's Waltz":
        d_arcs = e[2][e[1]]
        print(title)
        print(d_arcs)
        break

Loading data...
Done loading data. 0 out of 150 pieces were discarded because of errors.
Booker's Waltz
tensor([[ 3,  1],
        [ 3,  2],
        [ 4,  3],
        [ 6,  5],
        [ 7,  6],
        [ 8,  0],
        [ 8,  4],
        [ 8,  7],
        [10,  9],
        [11, 10],
        [13,  8],
        [13, 11],
        [13, 12]])


In [71]:
dtree2unlabeled_ctree(d_arcs.tolist()).unlabeled_repr()

[[0, [[[1, [2, 3]], 4], [[[5, 6], 7], 8]]], [[[9, 10], 11], [12, 13]]]

In [16]:
def get_all_spans(tree):
    all_spans = []
    def _recursive_get_all_spans(node):
        # if node is a leaf, end recursion
        if type(node) != list:
            return 
        else:
            # we traverse tree1, and match string with tree2             
            for child in node:
                _recursive_get_all_spans(child)
            # add information of the current node
            # compute the span of the subtree rooted at the current node
            digits = [i for i in re.split(r'(\D+)', str(node)) if i.isdigit()]
            spans = (digits[0], digits[-1])
            all_spans.append(spans)
        
    _recursive_get_all_spans(tree)
    return all_spans

def ctree_span_similarity(truth, pred, return_intersection=False):
    # compute all spans of truth and pred
    spans_truth = get_all_spans(truth)
    spans_pred = get_all_spans(pred)
    
    # find matching spans
    set_span_pred = set(spans_pred) # for computational reason, faster if it's a set
    intersection = [span for span in spans_truth if span in set_span_pred]
    
    if return_intersection:
        return len(intersection)/len(spans_truth), intersection
    else:
        return len(intersection)/len(spans_truth)


tree1 =  [[1,2],[2,3]]
tree2 = [[1,2],3]
similarity = ctree_span_similarity(tree1,tree2)

similarity

0.6666666666666666

[1, 2]

In [8]:
from collections import defaultdict
import re

tree1 =  [[1,22],3]
digits = [i for i in re.split(r'(\D+)', str(tree1)) if i.isdigit()]
spans = (digits[0], digits[-1])
spans

('1', '3')

In [20]:
a = [3,1,2]
a.sort()
a

[1, 2, 3]

# Check c-tree algorithms from an already trained model

In [1]:
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import Trainer, seed_everything
import torch
import random
import argparse
import warnings
warnings.filterwarnings('ignore') # avoid printing the partitura warnings
import os

/share/home/francesco/miniconda3/envs/musicparser/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import wandb
run = wandb.init()
artifact = run.use_artifact('fosfrancesco/Parsing JTB/model-zvxdt6sm:v0', type='model')
artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: fosfrancesco. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


In [3]:
from musicparser.data_loading import JTBDataModule
from musicparser.models import ArcPredictionLightModel

In [4]:
datamodule = JTBDataModule(batch_size=1, num_workers=1, data_augmentation="preprocess", only_tree=True)
datamodule.setup()

pos_weight = int(datamodule.positive_weight)
print("Using pos_weight", pos_weight)
input_dim = 64
model = ArcPredictionLightModel.load_from_checkpoint(checkpoint_path=os.path.join(os.path.normpath(artifact_dir), "model.ckpt"))

wandb_logger = True

trainer = Trainer(
    max_epochs=200, accelerator="auto", devices= [0], #strategy="ddp",
    num_sanity_val_steps=1,
    logger=wandb_logger,
    deterministic=True
    )

# trainer.tune(model, datamodule=datamodule)
# print("LR set to", model.lr)
trainer.predict(model, dataloaders=datamodule.test_dataloader())

Loading data...
Done loading data. 0 out of 150 pieces were discarded because of errors.
Augmenting data...
Augmenting data...
Train size :1440, Val size :15, Test size :15
No pretraining data
Using pos_weight 26


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting: 0it [00:00, ?it/s]